In [1]:
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset, num2date
from matplotlib.colors import LogNorm
from matplotlib import cm
import matplotlib
from scipy import stats
import copy
from function import confid

In [2]:
# data for fft of running avergaed filtered ssh
path = '/vortexfs1/home/wwu/Rossby_Wave_Project/data/fft_resample_hann_detrend_ssh_14yrs_all_freqs.nc'
#path = 'fft_filtered_hann_detrend_ssh_14yrs_all_freqs.nc'
nc = Dataset(path,'r')
lon,lat = np.meshgrid(nc.variables['lon'][:],nc.variables['lat'][:])
lon =360*(lon<0)+lon
fft_h = nc.variables['fftreal'][150:160,:,:] \
       + 1j* nc.variables['fftimg'][150:160,:,:]
N = 1022. # number of data points (use later to normalize fft output)
# the spacing of frequency 
df = nc.variables['cycle_per_day'][1]- nc.variables['cycle_per_day'][0]




# data for fft of 0.1 degree resolution SU,SV (depth-averaged U,V in cm/s)
path2= '/vortexfs1/home/wwu/Rossby_Wave_Project/data/fft_su_hann_33_day.nc'
path3= '/vortexfs1/home/wwu/Rossby_Wave_Project/data/fft_sv_hann_33_day.nc'
#path2 = 'data/fft_su_hann_33_day.nc'
#path3 = 'data/fft_sv_hann_33_day.nc'
nc2 = Dataset(path2,'r')
fft_u = nc2.variables['fft_su_real'][:] \
       + 1j* nc2.variables['fft_su_img'][:]
nc3 = Dataset(path3,'r')
fft_v = nc3.variables['fft_sv_real'][:] \
       + 1j* nc3.variables['fft_sv_img'][:]
R0 = 1.027 # sea water density in g/cm^3
# SU, SV are in unit of cm/s
S_u = np.mean(2.* (np.absolute(fft_u)**2.)/(N**2. * df),axis = 0)
S_v = np.mean(2.* (np.absolute(fft_v)**2.)/(N**2. * df),axis = 0)
#S_ke = 0.5 * R0 *(S_u + S_v) 

loc = np.empty(2,) # index of reference point
loc[0] = 354
loc[1] = 1100

In [3]:
fft_ref = fft_h[:,int(loc[0]),int(loc[1])]
S_ref = np.mean(2.*(np.absolute(fft_ref)**2.)/(N**2. * df),axis = 0)


cp = fft_u*np.conjugate(fft_ref)[...,np.newaxis,np.newaxis]
C_lf = np.mean(2.*cp/(N**2. * df),axis = 0)
coh_u = np.absolute(C_lf)**2./(S_u*S_ref)

cp = fft_v*np.conjugate(fft_ref)[...,np.newaxis,np.newaxis]
C_lf = np.mean(2.*cp/(N**2. * df),axis = 0)
coh_v = np.absolute(C_lf)**2./(S_v*S_ref)

frac = (coh_u*S_u + coh_v*S_v)/ (S_u + S_v)

In [ ]:
fig = plt.figure(dpi = 300)
degree_sign= u'\N{DEGREE SIGN}'
current_cmap = copy.copy(plt.cm.get_cmap('viridis'))
current_cmap.set_bad(color='0.6')
#bar = np.linspace(0,1,200)
cs = plt.pcolormesh(lon,lat,frac,shading = 'nearest',\
                    vmin = 0,vmax= 1.0,cmap = current_cmap)
plt.axis('scaled')
cb = plt.colorbar(cs,ticks = [0,0.5,1.0],fraction = 0.022)
cb.set_label('Fraction',fontsize = 10)

x0 = lon[int(loc[0]),int(loc[1])]
y0 = lat[int(loc[0]),int(loc[1])]
plt.plot(x0, y0, marker='^',color='red',markersize = 4)


plt.xticks([120,150,180,210,240,270],['120'+degree_sign+'E', '150'+degree_sign+'E','180'+degree_sign, '150'+degree_sign+'W','120'+degree_sign+'W','90'+degree_sign+'W'],fontsize = 8)
plt.yticks([-20,0,20,40,60],['20'+degree_sign+'S','0'+degree_sign, '20'+degree_sign+'N', '40'+degree_sign+'N', '60'+degree_sign+'N'],fontsize = 8)
plt.title('Fraction of HKE explained by SSH reference at '+\
          str(round(360-x0,1))+degree_sign+'W,'+str(int(y0))+degree_sign+'N',size =10.)


plt.savefig('HKE_fraction.png',dpi = 300,bbox_inches='tight', pad_inches=0)


plt.show()